In [ ]:
%matplotlib inline


# Fixed Point Arithmetics

In this tutorial we demonstrate we can evaluate the FIR filter using
fixed point arithmetics.


In [ ]:
import scipy.signal
import numpy as np
import cbadc
import matplotlib.pyplot as plt

## Setting up the Analog System and Digital Control

In this example, we assume that we have access to a control signal
s[k] generated by the interactions of an analog system and digital control.
Furthermore, we a chain-of-integrators converter with corresponding
analog system and digital control.

<img src="file://images/chainOfIntegratorsGeneral.svg" width="500" align="center" alt="The chain of integrators ADC.">



In [ ]:
# Setup analog system and digital control

# We fix the number of analog states.
N = 6
M = N
# Set the amplification factor.
beta = 6250.0
rho = -1e-2
kappa = -1.0
# In this example, each nodes amplification and local feedback will be set
# identically.
betaVec = beta * np.ones(N)
rhoVec = betaVec * rho
kappaVec = kappa * beta * np.eye(N)

# Instantiate a chain-of-integrators analog system.
analog_system = cbadc.analog_system.ChainOfIntegrators(betaVec, rhoVec, kappaVec)


T = 1 / (2 * beta)
digital_control = cbadc.digital_control.DigitalControl(T, M)


# Summarize the analog system, digital control, and digital estimator.
print(analog_system, "\n")
print(digital_control)

### Fixed Point Configuration

Next we configure the fixed point by instantiating
:py:class:`cbadc.utilities.FixedPoint`.




In [ ]:
bits_used = 20
max_floating_point_value = 1.0
fixed_point = cbadc.utilities.FixedPoint(bits_used, max_floating_point_value)
print(fixed_point)

### Impulse Response

Next we instantiate a :py:class:`cbadc.digital_estimator.FIRFilter` and
visualize its impulse responses.

Note the truncation in the impulse response as a result of the
:py:class:`cbadc.utilities.FixedPoint` configuration.



In [ ]:
# Choose an eta2 according to OSR
OSR = 1 << 5
omega_3dB = 2 * np.pi / (2 * T * OSR)
eta2 = (
    np.linalg.norm(analog_system.transfer_function_matrix(np.array([omega_3dB]))) ** 2
)

# Instantiate digital estimator
K1 = 1 << 8
K2 = 1 << 8
digital_estimator = cbadc.digital_estimator.FIRFilter(
    analog_system, digital_control, eta2, K1, K2, fixed_point=fixed_point
)

# extract impulse response
impulse_response = np.abs(np.array(digital_estimator.h[0, :, :]))

# Visualize the impulse response
h_index = np.arange(-K1, K2)
fig, ax = plt.subplots(2)
for index in range(N):
    ax[0].plot(h_index, impulse_response[:, index], label=f"$h_{index + 1}[k]$")
    ax[1].semilogy(h_index, impulse_response[:, index], label=f"$h_{index + 1}[k]$")
ax[0].legend()
fig.suptitle(f"For $\eta^2 = {10 * np.log10(eta2)}$ [dB]")
ax[1].set_xlabel("filter tap k")
ax[0].set_ylabel("$| h_\ell [k]|$")
ax[1].set_ylabel("$| h_\ell [k]|$")
ax[0].set_xlim((-50, 50))
ax[0].grid(which="both")
ax[1].set_xlim((-K1, K2))
ax[1].grid(which="both")

print(
    f"Total number of filter coefficients = {digital_estimator.number_of_filter_coefficients()}"
)

## Fixed Point Arithmetic Precision





In [ ]:
fixed_point_precision = np.arange(8, 17)

control_signal_sequences = [
    cbadc.utilities.byte_stream_2_control_signal(
        cbadc.utilities.read_byte_stream_from_file(
            "../a_getting_started/sinusodial_simulation.adcs", M
        ),
        M,
    )
    for _ in fixed_point_precision
]

size = 1 << 14
u_hat = np.zeros(size)

fixed_points = [cbadc.utilities.FixedPoint(bits, 2.0) for bits in fixed_point_precision]


digital_estimators = [
    cbadc.digital_estimator.FIRFilter(
        analog_system, digital_control, eta2, K1, K2, fixed_point=fixed_point,
    )
    for fixed_point in fixed_points
]

for index, bits in enumerate(fixed_point_precision):
    print(
        f"Precision = {bits} bits, number of non-zero filter coefficients = {digital_estimators[index].number_of_filter_coefficients()}"
    )


for index, de in enumerate(digital_estimators):
    de(control_signal_sequences[index])


for m in range(M):
    plt.figure()
    for bits, de in enumerate(digital_estimators):
        plt.semilogy(
            np.arange(0, K2),
            np.abs(np.array(de.h[0, :, :]))[K2:, m] * fixed_points[bits].min(),
            label=f"#bits = {fixed_point_precision[bits]}",
        )
    plt.legend()
    plt.xlabel("filter tap $k$")
    plt.ylabel(f"$h_[k, {m}]" + " / \text{max float}$")
    plt.xlim((0, K2))
    plt.grid(which="both")

digital_estimators_ref = cbadc.digital_estimator.FIRFilter(
    analog_system, digital_control, eta2, K1, K2
)

digital_estimators_ref(
    cbadc.utilities.byte_stream_2_control_signal(
        cbadc.utilities.read_byte_stream_from_file(
            "../a_getting_started/sinusodial_simulation.adcs", M
        ),
        M,
    )
)

for index in range(size):
    u_hat[index] = next(digital_estimators_ref)
f_ref, psd_ref = cbadc.utilities.compute_power_spectral_density(u_hat)


u_hats = []
fig, ax = plt.subplots(fixed_point_precision.size, 1)
plt.rcParams["figure.figsize"] = [6.40, 6.40 * 4]
for index_de, bits in enumerate(fixed_point_precision):
    # Compute estimates for each estimator
    for index in range(size):
        u_hat[index] = next(digital_estimators[index_de])
    u_hats.append(np.copy(u_hat))

    # Compute power spectral density
    f, psd = cbadc.utilities.compute_power_spectral_density(u_hat[K1:])
    signal_index = cbadc.utilities.find_sinusoidal(psd, 50)
    if (signal_index == psd.size).any():
        signal_index = np.arange(10)
    noise_index = np.ones(psd.size, dtype=bool)
    noise_index[signal_index] = False
    noise_index[0:2] = False
    noise_index[size // OSR :] = False
    res = cbadc.utilities.snr_spectrum_computation_extended(
        psd, signal_index, noise_index, fs=fs
    )
    SNR = 10 * np.log10(res["snr"])
    ENOB = np.round((SNR - 1.76) / 6.02, 1)
    print(f"SNR {round( SNR, 2)} dB, ENOB {ENOB}, fixed point precision = {bits}\n")

    # Plot the FIR filters
    color = next(ax[index_de]._get_lines.prop_cycler)["color"]

    ax[index_de].grid(b=True, which="major", color="gray", alpha=0.6, lw=1.5)
    ax[index_de].grid(b=True, which="major", color="gray", alpha=0.6, lw=1.5)

    ax[index_de].semilogx(f_ref, 10 * np.log10(psd_ref), label="Reference", color="k")

    ax[index_de].semilogx(f, 10 * np.log10(psd), label=f"bits={bits}", color=color)

    ax[index_de].set_ylabel("$ \mathrm{V}^2 \, / \, \mathrm{Hz}$")

    ax[index_de].legend()
    ax[index_de].set_xlim((0.0002, 0.5))

ax[-1].set_xlabel("frequency [Hz]")
fig.tight_layout()